In [ ]:
# When using a colab notebook:
#!git clone https://github.com/Aleph-Alpha/examples.git
#!pip install -r examples/requirements.txt
#!cp examples/bootcamp/data.md data.md

# Notebook 1: Completions
In this notebook we will learn how to use the API to create completions.

Follow the instructions below to get started!

Whenever there is a TODO, you will need to fill in the code to complete the task.

In [ ]:
from aleph_alpha_client import Client, Prompt, CompletionRequest, CompletionResponse
from scipy import spatial
import numpy as np
import os
from dotenv import load_dotenv
from langchain import PromptTemplate, LLMChain


from langchain.llms import AlephAlpha

### Step 0: Define some helper functions
Here we initialize the client and submit our Token in order to authenticate when using the API.

You can create a new Token in the playground. Visit https://app.aleph-alpha.com/profile to create a new API token.

When working locally, you can add it to the .env file in the root directory of the project.

Otherwise you can set it as the token as a string. E.g.: 
    
```python
client = Client(token="YOUR_TOKEN")
``````

In [ ]:
load_dotenv()

# for using the Aleph Alpha API
client = Client(token=os.getenv("AA_TOKEN"))

# for using LangChain
aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), model="luminous-extended")

#### Step 1: Use the completion function of the API
When calling the API, we want to define the parameters of how Luminous should solve the task.

The API has a completion function that takes a task and a set of parameters as input and returns a completion as output.

The completion function is called `complete` and is part of the `luminous` API module.


In this case we already provided the prompt for you. Your task is to define the parameters for the completion function.

You can find the documentation for the completion function [here](https://docs.aleph-alpha.com/docs/tasks/complete/).

Feel free to play around with the parameters and see how they affect the completion.

In [ ]:
# using the full-fledged AA API for a simple completion

# Define the prompt
prompt = Prompt.from_text("""### Instructions: Complete the following sentence with a few words.

### Input: An apple a day

### Response:""")

# Create a completion request with parameters
request = None # TODO: create a completion request with the prompt and parameters https://docs.aleph-alpha.com/docs/tasks/complete/ 

# Send the request to the API
response = client.complete(request=request, model="luminous-base-control")

# Get the completion
completion = response.completions[0].completion
print(completion)

---------------------------
Great Job! You've completed the first lesson!
Now let's try to do the same thing using `Langchain`.
Langchain is a library that provides a common interface to iinteract with different models and services.
It is designed to be easy to use and to be easily extensible.

You can find the documentation [here](https://langchain.readthedocs.io/en/latest/).

In [ ]:
# using LangChain for a simple completion

# define the prompt
prompt = "An apple a day"

# define the parameters
params = {
    "temperature": 0.5,
    "model": "luminous-extended",
    "maximum_tokens": 100
}

# define the model
aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), **params)

# get the completion
response = aleph_alpha(prompt=prompt, stop=["\n"])
print(response)

#### Step 2: Write a few shot prompt that generates keywords

Now that we know how to use the API, let's write a few shot prompt that generates keywords. 

First we will read in some text from a file. Then we will use the API to generate keywords from that text. Finally, we will print out the keywords.

##### Let's import some data to work on

In [ ]:
# Read the data in the data.md file
with open("data.md", "r", encoding="utf-8") as f:
    data = f.read()
    
# Split the data into a list of texts
texts = data.split("#")

# remove the first element of the list
texts = texts[1:]

print(f"data: {data[:100]}")

----------------
#### Let's create Keywords for the first rows of the dataset

We've already provided the function itself. Now you need to write a good prompt in order to get the desired result.

Try to either use `Few-shot` or `Zero-shot` learning.

You can find some tips on prompting the models here: 

https://docs.aleph-alpha.com/docs/introduction/zero-shot_prompting/

https://docs.aleph-alpha.com/docs/introduction/few_shot_prompting/

You can use either the Aelph Alpha Client or Langchain

In [ ]:
# Get the first 10 texts
texts_for_keywords = texts

# Let's write a function that takes a text and returns a list of keywords
def get_keywords(text):
    # TODO Write a prompt that generates keywords for any text
    # Tipp: Use few-shot learning
    
    # 1. TODO define the prompt
    prompt = Prompt.from_text(f"""A good Keyword prompt for {text}""")   
    
    # 2. define the completion request 
    request = CompletionRequest(prompt=prompt, 
                                    maximum_tokens=32, 
                                    temperature=0, 
                                    stop_sequences=["\n"])
    
    # 3. send the request to the API
    response = client.complete(request=request, model="luminous-base")
    
    # 4. get the completion
    completion = response.completions[0].completion
    return completion

# Alternatively you can write the function with langchain
def get_keywords_langchain(text):
    # TODO Write a prompt that generates keywords for any text
    # Tipp: Use few-shot learning
    
    # 1. TODO define the prompt
    prompt = f"""Prompt"""
    
    # 2. define the parameters
    params = {
        "temperature": 0.5,
        "model": "luminous-base-control",
        "maximum_tokens": 100
    }

    # 3. define the model
    aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), **params)
    
    # 4. get the completion
    response = aleph_alpha(prompt=prompt, stop=["\n"])
    return response
    

for text in texts:
    keywords = get_keywords(text)
    print(keywords)

------------------
### Step 3: Let's use Langchain to run a chain

Let's see how we can use Langchain to answer a question about a context.

In this example we use a LLMChain and provide a context and a question. The model will then try to answer the question based on the context.

Its your job to write a prompt that works well with the model.

Use `context` and `question` to create a prompt that will answer the question.

In [ ]:
# use a chain from langchain to generate a text


template = """{context}, {question}"""

prompt = PromptTemplate(template=template, input_variables=["question", "context"])

llm = AlephAlpha(model="luminous-base-control", maximum_tokens=32, stop_sequences=["###:"], aleph_alpha_api_key=os.getenv("AA_TOKEN"))

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "When was the declaration signed?"

answer = llm_chain.run(question = question, context = texts[1])

print(answer)

------------
### Step 4: generate Questions and Answers

In this section we will use luminous sequentially to generate questions and answers for the given text. We will use the `generate_questions_and_answers` function to generate questions and answers for the given text. 

Your job is to write bothe the question generation prompt as well as the answer generation prompt.

Feel free to use a previous prompt if you see fit.

In [ ]:
# TODO write two prompts one, that generates a question about a text and one that then answers the questions

def generate_questions_and_answers(text):
    # TODO Write a prompt that generates questions and answers for any text
    # Tipp: Use few-shot learning
        
    # 1. TODO define the prompt
    questions_prompt = Prompt.from_text(f"""prompt""")
    
    # 2. TODO define the completion request for the questions
    request = CompletionRequest(prompt=questions_prompt, temperature=0.0, stop_sequences=["\n"])
    response = client.complete(request=request, model="luminous-base-control")
    question = response.completions[0].completion
    
    # 3. TODO get the completion for answers
    answers_prompt = Prompt.from_text(f"""prompt""")   
    request = CompletionRequest(prompt=answers_prompt, temperature=0.0, maximum_tokens=128)
    response = client.complete(request=request, model="luminous-base-control")
        
    # 4. get the completion
    completion = response.completions[0].completion
    
    return question, completion

text = texts[0]

question, completion = generate_questions_and_answers(text)
print(f"Question: {question}")
print(f"Answer: \n{completion}")

---------------
### Step 5: Chatting with Luminous

Finally, let's write a function so that we can chat with Luminous.
We provided a lot of the code below.

Again you must find a good prompt to make Luminous say something interesting.

**Hint**: We are using the ``history`` parameter to hold the state with the previous messages. You can use this to make Luminous remember things you said in the past.

Please keep in mind that `Luminous` is not optimzed for chat and will not be able to hold a conversation for long. It is just a fun example of how you can use it.

In [ ]:
# Let's write a function where the user chats with an AI, store the conversation in a list as a memory
history = []
def chat_with_ai(message):
    # TODO Write a prompt that generates questions and answers for any text
    # Tipp: Use few-shot learning
    history.append(f"User: {message}")
    breaker = "\n"
    # 1. TODO try varyinf the prompt
    prompt = Prompt.from_text(f"""### Instructions: ...

### History: 
{breaker.join(history)}

### AI:""")
    
    # 2. TODO define the completion request for the answer
    request = CompletionRequest(prompt=prompt, temperature=0.5,)
    response = client.complete(request=request, model="luminous-extended-control")
    answer = response.completions[0].completion
    
    
    
    history.append(f"AI: {answer}")
    
    return "\n".join(history)


In [ ]:
print(chat_with_ai("Maybe you can help me out. What are the most importnat things to know about boats?"))